In [43]:
# import glob
# import json

import xml.etree.ElementTree as ET
# from bs4 import BeautifulSoup
# from tqdm import tqdm

import translators.server as tss

import re
from nltk.corpus import stopwords

import pandas as pd

In [44]:
# create name for variable
def clean_name(s, convert_to_lower=True):
    s = re.sub('<[^<]+?>', '', s) #markup
    s = re.sub('[^0-9a-zA-Z_\s]', '', s) #keep alnum
    s = re.sub('\t\n\r', '', s) #remove tab, line break, carriage return
    s = ' '.join(s.split()) #remove redundant whitespace
    return s.lower() if convert_to_lower else s

# remove unnecessary part in value
def clean_phrase(s, convert_to_lower=False):
    if(s is None or s == ''):
        return None
    s = re.sub('\t\n\r', '', s) #remove tab, line break, carriage return
    s = ' '.join(s.split()) #remove redundant whitespace
    return s.lower() if convert_to_lower else s

In [45]:
# create variable
def make_var(phrase, signature='_', keep_n_words=4):
    if (phrase is None or phrase == ''):
        return ''
    the_clean_phrase = clean_phrase(phrase) 
    # print(the_clean_phrase)
    if the_clean_phrase in var_phrase_map:
        return var_phrase_map[the_clean_phrase]
    else:
        h = signature + '_' + '_'.join([word for word in clean_name(phrase).split(' ') if word not in STOPWORDS][:keep_n_words])
        v = '%(' + str(h) + ')%'
        # key: value of the variable, value: name of the variable
        var_phrase_map[the_clean_phrase] = v
        # print('v', v)
        return v

# change variable name
def change_var(old_name, signature='_', keep_n_words=4):
    if old_name in var_name_map:
        return var_name_map[old_name]
    else:
        phrase = table_new.loc[old_name].iloc[0]
        # print(phrase)
        if (old_name is None or old_name == '' or pd.isnull(phrase)):
            return ''
        the_clean_phrase = clean_phrase(phrase) 
        # print(the_clean_phrase)
        h = signature + '_' + '_'.join([word for word in clean_name(phrase).split(' ') if word not in STOPWORDS][:keep_n_words])
        v = '%(' + str(h) + ')%'
        var_name_map[old_name] = v
        print(v)
        # key: value of the variable, value: name of the variable
        table_new.rename(index={old_name:v}, inplace=True)
        # print('v', v)
        return v

In [46]:
# find hash
def find_hash(s):
    if(s is None or s == ''):
        return False
    s = re.sub('%\(', '', s)
    s = re.sub('\)%', '', s)
    return s.lstrip('-').isdigit()

In [47]:
# replace pharse with variable
def iterate_generic(tag: str, root):
    """txt should be %% type or a pharse."""
    table_new_index_list = table_new.index.tolist() 
    count = 1
    for element in root.iter(tag):
        if tag == 'Input' and count > 1:
            continue
        # extract the txt from brd
        if tag == 'Input':
            txt = element[0].text
        else:
            txt = element.text
        # if txt is already in the mass production table and it is not a hash-like or txt is empty, we don't need to create a name
        if clean_phrase(txt) is None:
            continue
        elif txt in table_new_index_list and find_hash(txt) is False:
            continue
        # elif txt is already in the mass production table and it is a hash-like
        elif txt in table_new_index_list and find_hash(txt) is True:
            if tag == 'Input':
                element[0].text = change_var(txt, signature=tag+'_'+str(count))
            else:
                # print(txt)
                element.text = change_var(txt, signature=tag+'_'+str(count))
            # print(txt, find_hash(txt))
            # continue
        # # else create a variable name for the value
        # else:
        #     if tag == 'Input':
        #         element[0].text = make_var(txt, signature=tag+'_'+str(count))
        #     else:
        #         # print(txt)
        #         element.text = make_var(txt, signature=tag+'_'+str(count))
        count += 1

In [48]:
# read the tags and call all functions above
def process_file(infile_brd, infile_table, outfile_table, outfile_brd):
    global tree; tree = ET.parse(infile_brd)
    print("mass production brd input read")
    print("path: " + infile_brd)
    root = tree.getroot()
    global var_phrase_map; var_phrase_map = dict()
    global var_name_map; var_name_map = dict()

    global table_new; table_new = pd.read_csv(infile_table, sep="\t", index_col=0)
    print("mass production table input read")
    print("path: " + infile_table)

    tags = ['hintMessage', 'buggyMessage', 'successMessage', 
            'label', 'Input']
    for tag in tags:
        iterate_generic(tag, root) 

    # # create new dataframe and concat it with the latest mass production table
    # df_new = pd.DataFrame(var_phrase_map.keys(), index = list(var_phrase_map.values()))
    # df_dup = pd.concat([df_new.T]*len(table_new.columns)).T
    # df_dup.columns = table_new.columns
    # df_mix = pd.concat([table_new, df_dup])

    # # export the csv
    # df_mix.to_csv(outfile_table, encoding="utf-8", sep="\t")
    # print("mass production table output finished")
    # print("path: " + outfile_table)

    # # export the brd
    # tree.write(outfile_brd)
    # print("mass production brd output finished")
    # print("path: " + outfile_table)

    return

In [49]:
# # # %(4060445299635058035)%
# infile_table = "./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new.txt"
# table_new = pd.read_csv(infile_table, sep="\t", index_col=0)
# table_new.head()

In [50]:
# table_new.loc['%(-3153624248818871842)%'].iloc[0]
# table_new.loc["%(6881591536898152652)%"].iloc[0]

# table_new.rename(index={'%(Part)%':'new_name'}, inplace=True)
# table_new

In [51]:
# read the latest mass production graph
infile_brd = "./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new.brd"
STOPWORDS = stopwords.words('english')

# read the latest mass production table
infile_table = "./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new.txt"

# set the output paths
# outfile_table = './Output_cleaned_folder/6.5_cleaned.txt'
# outfile_brd = "./Output_cleaned_folder/6.5_cleaned.brd"
outfile_brd = infile_brd.replace('/HTML_folder/6.05 HTML/6.05 HTML/MassProduction/', '/Output_cleaned_folder/').replace('.brd', '_cleaned.brd')
outfile_table = infile_table.replace('/HTML_folder/6.05 HTML/6.05 HTML/MassProduction/', '/Output_cleaned_folder/').replace('.txt', '_cleaned.txt')

# run the process function
process_file(infile_brd, infile_table, outfile_table, outfile_brd)

mass production brd input read
path: ./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new.brd
mass production table input read
path: ./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new.txt
%(hintMessage_1_many_pictures_kelly_give)%
%(hintMessage_2_read_problem_statement_find)%
%(hintMessage_4_please_enter_32_highlighted)%
%(hintMessage_5_many_senior_pictures_kelly)%
%(hintMessage_6_read_problem_statement_find)%
%(hintMessage_7_please_enter_50_highlighted)%
%(hintMessage_8_number_senior_pictures_representing)%
%(hintMessage_10_please_select_total_number)%
%(hintMessage_11_enter_lowercase_letter_represent)%
%(hintMessage_12_choose_letter_z)%
%(hintMessage_13_please_enter_n_highlighted)%
%(hintMessage_14_look_information_gathered_question)%
%(hintMessage_15_denominator_multiple_factor_one)%
%(hintMessage_16_since_denominator_factor_multiple)%
%(hintMessage_17_please_select_cross_multiplication)%
%(hintMessage_20_since_denominator_multiple_100)%

In [52]:
# table_new.iloc[69]